In [43]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import Normalizer, StandardScaler

import pandas as pd
import numpy as np

In [84]:
df_train = pd.read_csv("train.csv", indeX_col=0)
df_train.shape

(1107611, 28)

In [85]:
X=df_train.drop(["target"], aXis=1)
y=df_train["target"]

#### Значения, которые точно нужно дробнуть

lei - Legal Entity Identifier. Идентификатор пользователя. Шум
county_code - Вроде не самый бесполезный параметр, но там слишком много различных значений, тоже больше похоже на шум


#### Значения, которые вероятно нужно дробнуть

county_code - Вроде не самый бесполезный параметр, но там слишком много различных значений, тоже больше похоже на шум
census_tract - Номер райончика. Тоже самое, слишком много значений. Нужно поэксперементировать 

### Важные значния, которые нужно переделать из категориальных
- conforming_loan_limit - Не до конца понял, что означают возможные значнения(C/NC/U), но это что-то вроде готовности государственных компаний выкупить эту ипотеку
- derived_loan_product_type - что-то про возможность спереть дома, пока кредит не будет выплочен, ну либо просто спереть дом. Так-то связано с lien_status, может быть их как-то объеденить.
- derived_dwelling_category - чето про категорию проживания заемщика
- derived_ethnicity, derived_race - стоит заюзать, но объеденить в одну категорию 
- derived_seX

*Эти интовые, но там какая-то лабуда в значениях, нужно наверно 1111 заменить на 0*
- reverse_mortage - финансовое соглашение, по которому домовладелец отказывается от доли в своем доме в обмен на регулярные выплаты, обычно в дополнение к пенсионному доходу. Не знаю повлияет или нет, но может быть. https://clck.ru/38S9F7 - картиночка, которая это понятно объясняет.
- open-end_line_of_credit - Открытый кредит — это кредит от банка или другого финансового учреждения, который заемщик может использовать повторно, до определенной заранее утвержденной суммы, и который не имеет фиксированной даты окончания полного погашения. Открытый кредит также называют возобновляемым кредитом.
- business_or_commercial_purpose


### Короче, все что я гуглил
- loan_to_value_ratio - (размер долга)/(стоимость покупки)
- total_loan_costs - сколько банк заработает на кредите - проценты и сборы, тело кредита не включается
- total_points_and_fees - короче, судя по описанию это что-то вроде первоначального взноса. Но почему-то на сайте фэнимэй написано про ограничение в 3 и в 5 процентов в зависимости от кейсов, мне лень разбираться в америкоском законодательстве. Короче, чем больше, тем лучше prolly, хотя вот здесь наверно стоит перевести это в проценты от самого кредита, потому что в таблице даны абсолютные значения. Для предыдущей ерунды то же самое
- origination_fee - плата за то, чтобы банк работать начал. Никак не покрывает дальнейший кредит
- discount_points - вот тоже про первоначальный взнос. Короче total_points_and_fees это видимо их сумма
- lender_credits - это discount_points задом наперед. Ты платишь меньше origination_fee, но у тебя больше месячная ставка. Короче, тебе типа в кредит закидывают оплату работы банка, мб еще работы юристов, с которыми ты дом покупал
- loan_term - продолжительность кредита, видимо в месяцах
- prepayment_penalty_term - сбор за то, что ты решишь выплатить весь свой кредит раньше срока
- intro_rate_period - период, пока ставка по кредиту ниже первоначальной
- negative_amortization - Иногда заемщик может временно платить меньше денег, чем ему начисляют процентов. Тогда кредит растет. Это называется отрицательной амортизацией
- interest_only_payment - тип ипотеки, когда заемщику какое-то время позволяется платить только проценты
- balloon_payment - тип ипотеки, когда заемщик платит маленькие выплаты в течении всего времени, а потом платит одну большую выплату в конце кредитного периода
- other_nonamortizing_features - Неамортизируемые кредиты требуют, чтобы их основная сумма выплачивалась единовременно, а не регулярными платежами, и обычно имеют короткий срок и высокую процентную ставку. Видимо имеется в виду такой тип кредитования. Потому что там тоже true/false значения

-сonstruction_method - там два типа данных 1 и 2. Понятия не имею, че они имеют в виду. Нужно построить графики, посмотреть есть ли вообще какая-то корреляция

-occupancy_type - основной дом, второй дом, инвестиция
-manufactured_home_secured_property_type - что-то про дома, которые можно произвести на заводе, а потом быстро собрать. Короче, довольно стандартные вот эти американские дома (https://en.wikipedia.org/wiki/Mobile_home)
-manufactured_home_land_ property_ interest - процент за землю для сборного дома


### Самая глупая базовая моделька

In [39]:
X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

In [40]:
model = LGBMClassifier(verbosity=-1, random_state=42, maX_depth=100, n_estimators=400)
model.fit(X_train, y_train)

LGBMClassifier(max_depth=100, n_estimators=400, random_state=42, verbosity=-1)

In [33]:
y_pred = model.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, y_pred)

0.924559602391559

In [53]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1107611 entries, 0 to 1107610
Data columns (total 28 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   applicant_ethnicity             1107611 non-null  float64
 1   income                          984345 non-null   float64
 2   applicant_race_1                1107376 non-null  float64
 3   applicant_race_2                35442 non-null    float64
 4   applicant_race_3                3369 non-null     float64
 5   applicant_race_4                374 non-null      float64
 6   applicant_race_5                106 non-null      float64
 7   applicant_sex                   1107611 non-null  int64  
 8   co_applicant_ethnicity          1107611 non-null  int64  
 9   co_applicant_race_1             1107523 non-null  float64
 10  co_applicant_race_2             11395 non-null    float64
 11  co_applicant_race_3             999 non-null      float64
 12  co_ap

#### Другая попытка оптимизации

In [91]:
X["applicant_race_1"] = X["applicant_race_1"].fillna(X["applicant_race_1"].median())
X["applicant_race_2"] = X["applicant_race_2"].fillna(X["applicant_race_2"].median())
X["applicant_race_3"] = X["applicant_race_3"].fillna(X["applicant_race_3"].median())
X["applicant_race_4"] = X["applicant_race_4"].fillna(X["applicant_race_4"].median())
X["applicant_race_5"] = X["applicant_race_5"].fillna(X["applicant_race_5"].median())

X["applicant_race_2"].isna().sum()

0

In [12]:


X = X.astype({
    "applicant_ethnicity": "category",
    "applicant_race_1": "category",
    "applicant_race_2": "category",
    "applicant_race_3": "category",
    "applicant_race_4": "category",
    "applicant_race_5": "category",
    "applicant_seX": "category",
    "co_applicant_ethnicity": "category",
    "co_applicant_race_1": "category",
    "co_applicant_race_2": "category",
    "co_applicant_race_3": "category",
    "co_applicant_race_4": "category",
    "co_applicant_race_5": "category",
    "co_applicant_seX": "category",


    "property_type": "category",
    
    "loan_purpose": "category",
    "loan_type": "category",
    "preapproval": "category",
    "property_type": "category",

})

X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1107611 entries, 0 to 1107610
Data columns (total 27 columns):
 #   Column                          Non-Null Count    Dtype   
---  ------                          --------------    -----   
 0   applicant_ethnicity             1107611 non-null  category
 1   income                          984345 non-null   float64 
 2   applicant_race_1                1107376 non-null  category
 3   applicant_race_2                35442 non-null    category
 4   applicant_race_3                3369 non-null     category
 5   applicant_race_4                374 non-null      category
 6   applicant_race_5                106 non-null      category
 7   applicant_sex                   1107611 non-null  category
 8   co_applicant_ethnicity          1107611 non-null  category
 9   co_applicant_race_1             1107523 non-null  category
 10  co_applicant_race_2             11395 non-null    category
 11  co_applicant_race_3             999 non-null      categ

In [101]:
X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

In [102]:
model = LGBMClassifier(verbosity=-1, random_state=42, maX_depth=100, n_estimators=400)
model.fit(X_train, y_train)

LGBMClassifier(max_depth=100, n_estimators=400, random_state=42, verbosity=-1)

In [103]:
y_pred = model.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, y_pred)

0.9243089917032655

### Первая попытка че-то улучшить

In [66]:
import lightgbm as lgb
def convert_to_lgb_dataset(df, cats):
    # Separate features and target variable
    features = df.drop('target', aXis=1)
    target = df['target']
    
    # Create LightGBM Dataset
    lgb_dataset = lgb.Dataset(data=features, label=target, categorical_feature=cats)
    lgb_dataset.construct()

    return lgb_dataset

In [ ]:
label = df_train.columns.to_numpy()

cats = ["applicant_ethnicity", "income", "applicant_race_1", "applicant_race_2", "applicant_race_3", "applicant_race_4", "applicant_race_5", "applicant_seX",       "co_applicant_ethnicity", "co_applicant_race_1", "co_applicant_race_2", "co_applicant_race_3", "co_applicant_race_4", "co_applicant_race_5", "co_applicant_seX", "loan_purpose", "loan_type"]

train_data = convert_to_lgb_dataset(df_train, cats)
train_data.num_feature()

In [68]:
param = {'metric': 'auc'}
num_round = 10

bst = lgb.train(param, train_data, num_round)

LightGBMError: Cannot set categorical feature after freed raw data, set free_raw_data=False when construct Dataset to avoid this.

### Подготовка самбишина

Код нужный, чтобы из модели получить файл для загрузки на кодалаб

In [112]:
X=pd.read_csv("test.csv", index_col=0)
sample_submission_df = pd.read_csv("submission.csv")

sample_submission_df.shape

(1107611, 2)

In [113]:
X["applicant_race_1"] = X["applicant_race_1"].fillna(X["applicant_race_1"].median())
X["applicant_race_2"] = X["applicant_race_2"].fillna(X["applicant_race_2"].median())
X["applicant_race_3"] = X["applicant_race_3"].fillna(X["applicant_race_3"].median())
X["applicant_race_4"] = X["applicant_race_4"].fillna(X["applicant_race_4"].median())
X["applicant_race_5"] = X["applicant_race_5"].fillna(X["applicant_race_5"].median())

In [114]:
pca = PCA(n_components=1)
kulin = X[["applicant_race_1", "applicant_race_2", "applicant_race_3", "applicant_race_4", "applicant_race_5"]]



pca.fit(kulin)
races = pca.transform(kulin)
X["applicant_race"] = races

X.drop(["applicant_race_1", "applicant_race_2", "applicant_race_3", "applicant_race_4", "applicant_race_5"], axis=1)

,applicant_ethnicity,income,applicant_sex,co_applicant_ethnicity,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,co_applicant_sex,...,preapproval,property_type,purchaser_type,hud_median_family_income,loan_amount,number_of_1_to_4_family_units,number_of_owner_occupied_units,minority_population,population,applicant_race
0,2,90.0,2,4,8.0,NaN,NaN,NaN,NaN,5,...,2,3,0,68300,55000.0,889,866,94.42,3282,-0.173805
1,2,77.0,2,2,5.0,NaN,NaN,NaN,NaN,1,...,2,3,0,76000,15000.0,2428,1828,19.71,6031,-0.173805
2,2,0.0,1,2,5.0,NaN,NaN,NaN,NaN,2,...,2,3,1,82100,275000.0,796,839,10.13,2320,-0.173805
3,3,NaN,4,3,7.0,NaN,NaN,NaN,NaN,4,...,2,3,2,84500,405000.0,2107,1883,48.07,7557,1.782376
4,2,84.0,2,4,8.0,NaN,NaN,NaN,NaN,5,...,2,3,3,68600,215000.0,2339,1764,16.55,5546,-0.173805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276898,3,245.0,4,3,7.0,NaN,NaN,NaN,NaN,4,...,2,3,0,60700,745000.0,1604,354,42.13,4253,1.782376
276899,2,52.0,1,4,8.0,NaN,NaN,NaN,NaN,5,...,2,3,0,79500,45000.0,2174,1482,18.85,4924,-7.411242
276900,2,140.0,1,2,5.0,NaN,NaN,NaN,NaN,2,...,2,3,71,82100,365000.0,2036,1142,25.43,4553,-4.086166
276901,2,69.0,1,4,8.0,NaN,NaN,NaN,NaN,5,...,2,3,2,84500,285000.0,2088,1470,62.29,8960,-2.129986


In [115]:
test_score = model.predict_proba(X)[:, 1]
test_score.shape

(276903,)

In [116]:
X["target"] = test_score

In [117]:
X.target.to_csv('submission.csv')

In [65]:
!zip submission.zip submission.csv

'zip' is not recognized as an internal or external command,
operable program or batch file.
